In [54]:

import numpy as np
import pandas as pd
import nltk
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import re
import random 
import collections
from gensim.models import KeyedVectors
# use multiprocessing for multicore ops

In [55]:
data1 = pd.read_csv("train.csv", delimiter=',', encoding="utf-8-sig")
data2 = pd.read_csv("test.csv", delimiter=',', encoding="utf-8-sig")
EMBEDDING_FILE = 'GoogleNews-vectors-negative300.bin.gz'
word2vec = KeyedVectors.load_word2vec_format(EMBEDDING_FILE, binary=True)
print(data1.shape)
print(data1.iloc[0:1,3:])
#random.shuffle(data1)

(404290, 6)
                                           question1  \
0  What is the step by step guide to invest in sh...   

                                           question2  is_duplicate  
0  What is the step by step guide to invest in sh...             0  


In [56]:
subset = data1.iloc[0:100,3:]
subset
#subset = subset.loc[subset.is_duplicate==0,['question1','question2']]


,question1,question2,is_duplicate
0,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0
5,Astrology: I am a Capricorn Sun Cap moon and c...,"I'm a triple Capricorn (Sun, Moon and ascendan...",1
6,Should I buy tiago?,What keeps childern active and far from phone ...,0
7,How can I be a good geologist?,What should I do to be a great geologist?,1
8,When do you use シ instead of し?,"When do you use ""&"" instead of ""and""?",0
9,Motorola (company): Can I hack my Charter Moto...,How do I hack Motorola DCX3400 for free internet?,0


In [57]:
#nltk.download()
import nltk
#nltk.download('stopwords')
from nltk.corpus import stopwords # Import the stop word list
print(stopwords.words("english")) 

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [58]:
def question_to_words( raw_quest ):
    lemmetizer = nltk.WordNetLemmatizer()
    letters_only = re.sub("[^a-zA-Z]", " ", raw_quest) 
    letters_only = letters_only.lower()
    words = nltk.word_tokenize(letters_only)
    #words = nltk.word_tokenize(s)
    stops = set(stopwords.words("english"))                  
    meaningful_words = [w for w in words if not w in stops]
    meaningful_words = [lemmetizer.lemmatize(w) for w in meaningful_words]
    return( " ".join( meaningful_words ))

In [59]:
#nltk.download('punkt')
#nltk.download('wordnet')
subset['question1']= subset['question1'].apply(question_to_words)
subset['question2']= subset['question2'].apply(question_to_words)

In [60]:
X = subset.iloc[:,0:-1]
X
y = subset.iloc[:,-1]



In [61]:
def text_to_word_list(text):
    ''' Pre process and convert texts to a list of words '''
    text = str(text)
    text = text.lower()

    # Clean the text
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "cannot ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r" u s ", " american ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)

    text = text.split()

    return text
stops = set(stopwords.words('english'))

In [62]:
vocabulary = dict()
inverse_vocabulary = ['<unk>']  # '<unk>' will never be used, it is only a placeholder for the [0, 0, ....0] embedding

questions_cols = ['question1', 'question2']

# Iterate over the questions only of both training and test datasets
for dataset in [data1, data2]:
    for index, row in dataset.iterrows():

        # Iterate through the text of both questions of the row
        for question in questions_cols:

            q2n = []  # q2n -> question numbers representation
            for word in text_to_word_list(row[question]):

                # Check for unwanted words
                if word in stops and word not in word2vec.vocab:
                    continue

                if word not in vocabulary:
                    vocabulary[word] = len(inverse_vocabulary)
                    q2n.append(len(inverse_vocabulary))
                    inverse_vocabulary.append(word)
                else:
                    q2n.append(vocabulary[word])

            # Replace questions as word to question as number representation
            dataset.set_value(index, question, q2n)
            
embedding_dim = 300
embeddings = 1 * np.random.randn(len(vocabulary) + 1, embedding_dim)  # This will be the embedding matrix
embeddings[0] = 0  # So that the padding will be ignored

# Build the embedding matrix
for word, index in vocabulary.items():
    if word in word2vec.vocab:
        embeddings[index] = word2vec.word_vec(word)

C:\Users\mvincho\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:28: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead


In [73]:
X = data1[questions_cols]
Y = data1['is_duplicate']
X_train, X_test, y_train, y_test = train_test_split(X, Y,test_size=0.2)
X_train

,question1,question2
101513,"[21, 99, 743, 193, 1302, 2689, 188, 1220, 212,...","[21, 99, 743, 446, 193, 1302, 1361, 188, 1, 19..."
112035,"[6095, 48376, 605, 671, 3028, 896, 218, 129, 3...","[26, 214, 640, 164, 16, 344, 225, 15138, 429, ..."
37123,"[4271, 70, 1, 2, 3, 664, 409, 211, 5525, 211, ...","[26, 97, 99, 3570, 1000, 11261, 409, 28889, 15..."
193597,"[26, 97, 16, 357, 8869, 79, 1, 1455, 300]","[26, 97, 16, 212, 3572, 79, 1, 1455, 300, 167,..."
68972,"[115, 307, 4912, 6593, 409, 283, 41, 167, 41, ...","[21, 3, 520, 15, 151, 4912, 2, 139, 83, 41, 97..."
182598,"[1, 115, 195, 744, 1188, 1186, 671, 674, 3950]","[56, 2, 195, 4586, 671, 3950]"
60474,"[76, 10804, 1250, 1101, 1257, 1252, 175, 1250,...","[26, 164, 2620, 1250, 1101, 1257, 1252, 357, 3..."
97461,"[41, 7197, 787, 185, 830, 147, 1733, 827]","[26, 538, 16, 455, 1733, 827]"
115271,"[41, 115, 335, 469, 3, 192, 412]","[41, 115, 335, 469, 192]"
350850,"[1, 2, 3, 629, 4712, 1093]","[550, 2, 9130, 225, 7977, 215, 3, 17482, 175, ..."


In [74]:

max_seq_length = max(X_train.question1.map(lambda x: len(x)).max(),
                     X_train.question2.map(lambda x: len(x)).max(),
                     X_test.question2.map(lambda x: len(x)).max(),
                    X_test.question2.map(lambda x: len(x)).max())

In [75]:
X_train = {'left': X_train.question1, 'right': X_train.question2}
X_test = {'left': X_test.question1, 'right': X_test.question2}

In [76]:
y_train = y_train.values
y_test = y_test.values

In [77]:
from time import time
import pandas as pd
import numpy as np
from gensim.models import KeyedVectors
import re
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns

import itertools
import datetime

from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from keras.layers import Input, Embedding, LSTM, Lambda
import keras.backend as K
from keras.optimizers import Adadelta
from keras.callbacks import ModelCheckpoint

In [78]:
max_seq_length
n_hidden = 50
gradient_clipping_norm = 1.25
batch_size = 64
n_epoch = 25

In [81]:
from keras.preprocessing.sequence import pad_sequences
for dataset, side in itertools.product([X_train, X_test], ['left', 'right']):
    dataset[side] = pad_sequences(dataset[side], maxlen=max_seq_length)

#Make sure everything is ok
assert X_train['left'].shape == X_train['right'].shape
assert len(X_train['left']) == len(y_train)

In [82]:
# Model variables
#!pip install keras
from keras.layers import Input, Embedding, LSTM, Lambda
n_hidden = 50
gradient_clipping_norm = 1.25
batch_size = 64
n_epoch = 25



def exponent_neg_manhattan_distance(left, right):
    ''' Helper function for the similarity estimate of the LSTMs outputs'''
    return K.exp(-K.sum(K.abs(left-right), axis=1, keepdims=True))

# The visible layer
left_input = Input(shape=(max_seq_length,), dtype='int32')
right_input = Input(shape=(max_seq_length,), dtype='int32')

In [ ]:
embedding_layer = Embedding(len(embeddings), embedding_dim, weights=[embeddings], input_length=max_seq_length, trainable=False)

# Embedded version of the inputs
encoded_left = embedding_layer(left_input)
encoded_right = embedding_layer(right_input)

# Since this is a siamese network, both sides share the same LSTM
shared_lstm = LSTM(n_hidden)

left_output = shared_lstm(encoded_left)
right_output = shared_lstm(encoded_right)

# Calculates the distance as defined by the MaLSTM model
malstm_distance = Lambda(function=lambda x: exponent_neg_manhattan_distance(x[0], x[1]),output_shape=lambda x: (x[0][0], 1))([left_output, right_output])

# Pack it all up into a model
malstm = Model([left_input, right_input], [malstm_distance])

# Adadelta optimizer, with gradient clipping by norm
optimizer = Adadelta(clipnorm=gradient_clipping_norm)

malstm.compile(loss='mean_squared_error', optimizer=optimizer, metrics=['accuracy'])

# Start training
training_start_time = time()

malstm_trained = malstm.fit([X_train['left'], X_train['right']], y_train, batch_size=batch_size, nb_epoch=n_epoch,
                            validation_data=([X_test['left'], X_test['right']], y_test))

print("Training time finished.\n{} epochs in {}".format(n_epoch, datetime.timedelta(seconds=time()-training_start_time)))

C:\Users\mvincho\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:28: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


Train on 323432 samples, validate on 80858 samples
Epoch 1/25
172160/323432 [==============>...............] - ETA: 8:47 - loss: 0.1842 - acc: 0.7296